In [ ]:
!pip install -q rasterio
! pip install -q rioxarray

     |████████████████████████████████| 47 kB 2.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
# Bibliotecas
from osgeo import gdal
import folium
import rasterio as rio
import numpy as np
import matplotlib.pyplot as plt
from folium import plugins

In [ ]:
# Definindo caminho de arquivos
img_dir = '/content/drive/MyDrive/Curso PDI com Python/'
img_path = img_dir + 'L71221071_07120010720_DN.tif'

In [ ]:
# Reprojetando imagem para WGS84
input_raster = gdal.Open(img_path)
output_raster = 'L71221071_07120010720_DN_wgs84.tif'
warp = gdal.Warp(output_raster,input_raster,dstSRS='EPSG:4326')
warp = None 

In [ ]:

# Função de expansão  do histograma
def expansao(img, percent_ini=2, percent_fim=98):
    s = np.zeros_like(img)
    x,y = 0,255  
    w = np.percentile(img, percent_ini)
    z = np.percentile(img, percent_fim)        
    p = x + (img - w) * (y - x) / (z - w)    
    p[p<x] = x
    p[p>y] = y
    s = p
    return s

In [ ]:
# Lendo arquivo reprojetado
with rio.open(output_raster) as src:
    b1 = src.read(1)
    b4 = src.read(4)
    b3 = src.read(3)

    img = np.dstack([b3,b4,b1])
        
    

# Calculando latitude e longitude central  
cent_lon = src.bounds[0] + (src.bounds[2] - src.bounds[0])/2
cent_lat = src.bounds[1] + (src.bounds[3] - src.bounds[1])/2

# Definindo basemap
m = folium.Map(location=[cent_lat,  cent_lon],
                   tiles='Stamen Terrain', zoom_start = 10)

# Adicionando imagem
m.add_child(folium.raster_layers.ImageOverlay(expansao(img), opacity=1, 
                                 bounds = [[src.bounds[1],src.bounds[0]],[src.bounds[3],src.bounds[2]]]))

# Visualizando resultado
m